In [ ]:
import numpy as np
from copy import copy

pour archive pcq j'ai pas confiance au git

In [ ]:
def untrace_tree(trace_path, node, before):
    if node.result is not None:
        before.append((None, node.result))
        trace_path.append(before)
        print(f"End of tree at {node.result}")
    else:
        before_false = copy(before)
        before_false.append((node.feature, False, node.value))
        before_true = copy(before)
        before_true.deepcopy((node.feature, True, node.value))
        untrace_tree(trace_path, node.false_branch, before_false)
        untrace_tree(trace_path, node.true_branch, before_true)


trace_path = []
untrace_tree(trace_path, tree_xai.tree, [])
trace_path

In [ ]:
def filter_trace_path(class_to_not_keep, trace_path):
    l = []
    for path in trace_path:
        end_class = path[-1][1]
        if end_class != class_to_not_keep:
            l.append(path)
    return l


filtered_trace_path = filter_trace_path(1, trace_path)
filtered_trace_path

In [ ]:
def count_rules(to_explain, trace_path):
    l = []
    for path in trace_path:
        n_r = 0
        for rule in path[:-1]:
            a, b, c = rule
            if b is False:
                if to_explain[a] != c:  # l'exemple valide la règle
                    pass
                else:  # l'exemple ne valise pas la règle
                    n_r += 1
            else:  # b == '=='
                if to_explain[a] == c:  # l'exemple valide la règle
                    pass
                else:  # l'exemple ne valise pas la règle
                    n_r += 1
        l.append(n_r)
    return np.array(l)


sample = X.iloc[0]
class_to_not_keep = y.iloc[0]
print(sample)
print(class_to_not_keep)
filtered_trace_path = filter_trace_path(1, trace_path)
n_r_list = count_rules(sample, filtered_trace_path)
print(n_r_list)
np.argwhere(n_r_list == np.amax(n_r_list)) ##pour méthode LORE ??

In [ ]:
sample, sample_label = X.iloc[0], y.iloc[0]

counterfacts = tree_xai.find_counterfactuals(sample_label)
counterfacts

n_r = tree_xai.count_rules(sample, counterfacts)
n_r

In [ ]:
@staticmethod
def format_path(path, result):
    """Formatte un chemin en un human-readable string représentant les règles de
    décision.
    """
    conditions = []
    for feature, decision, value in path:
        verbe = "est" if decision == "==" else "n'est pas"
        condition = f"'{feature}' {verbe} '{value}'"
        conditions.append(condition)
    if len(conditions) > 1:
        conditions_str = ", ".join(conditions[:-1]) + " et " + conditions[-1]
    else:
        conditions_str = conditions[0]
    return f"Si {conditions_str} alors classe {result}"